In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored

In [2]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antoniolopardo (use `wandb login --relogin` to force relogin)


True

In [3]:
import dataset_handler as dh
import helper_func as hf

In [17]:
torch.manual_seed(0)
np.random.seed(8)

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

"""Load the priming text to add to the prompt and sample a question"""
# priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt_codegen.txt"
# priming_text = read_string_from_file("data/priming_texts/singleEq.txt")
priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt"

current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(50)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Write a program that prints the answer to the following question. When they got there, they saw that SFL is already filled with people. Every entrance has a long line, with 283 people waiting to get in. If SFL has 5 entrances, how many people in total are waiting to get in?
1415 (people)


In [5]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 475.45s
loading tokenizer


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading tokenizer took 7.91s


In [18]:
def test(config=None):
    with wandb.init(config=config):
        config = wandb.config

        pass_at_k = hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

        wandb.log({"pass_at_k": pass_at_k})

In [19]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'pass_at_k',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'k': {
        'values': [3]
        },
    'do_sample': {
        'values': [True]
        },
    'top_k': {
          'values': [10, 1, 50, 100]
        },
    'top_p': {
          'values': [0.0, 0.2, 0.5, 0.9]
        },
    'temperature': {
          'values': [0.1, 0.2, 0.5, 0.9]
        },
    'min_length': {
          'values': [1]
        },
    'max_length_after_input': {
          'values': [100, 200, 250]
        },
    'num_return_sequences': {
          'values': [4]
        },
    }

sweep_config['parameters'] = parameters_dict

In [20]:
sweep_id = wandb.sweep(sweep_config, project="PracticalWork")
wandb.agent(sweep_id, test, count=15)

Create sweep with ID: aq1jys2a
Sweep URL: https://wandb.ai/antoniolopardo/PracticalWork/sweeps/aq1jys2a


wandb: Agent Starting Run: 7acjjotr with config:
wandb: 	do_sample: True
wandb: 	k: 3
wandb: 	max_length_after_input: 200
wandb: 	min_length: 1
wandb: 	num_return_sequences: 4
wandb: 	temperature: 0.1
wandb: 	top_k: 100
wandb: 	top_p: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TESTING STARTED
@sample 5 -> Pass@3 = 0.4
@sample 10 -> Pass@3 = 0.2
@sample 15 -> Pass@3 = 0.2857142857142857
@sample 20 -> Pass@3 = 0.2631578947368421
@sample 25 -> Pass@3 = 0.3333333333333333
@sample 30 -> Pass@3 = 0.3793103448275862
@sample 35 -> Pass@3 = 0.35294117647058826
@sample 40 -> Pass@3 = 0.38461538461538464
@sample 45 -> Pass@3 = 0.38636363636363635
@sample 50 -> Pass@3 = 0.3469387755102041


Pass@3 = 0.3469387755102041



pass_at_k,▁
pass_at_k,0.34694


wandb: Agent Starting Run: yr0ec77b with config:
wandb: 	do_sample: True
wandb: 	k: 3
wandb: 	max_length_after_input: 200
wandb: 	min_length: 1
wandb: 	num_return_sequences: 4
wandb: 	temperature: 0.1
wandb: 	top_k: 10
wandb: 	top_p: 0.2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

TESTING STARTED
@sample 5 -> Pass@3 = 0.4
@sample 10 -> Pass@3 = 0.2
@sample 15 -> Pass@3 = 0.2857142857142857
@sample 20 -> Pass@3 = 0.2631578947368421
@sample 25 -> Pass@3 = 0.3333333333333333
@sample 30 -> Pass@3 = 0.3793103448275862
@sample 35 -> Pass@3 = 0.35294117647058826
@sample 40 -> Pass@3 = 0.38461538461538464
@sample 45 -> Pass@3 = 0.38636363636363635
@sample 50 -> Pass@3 = 0.3469387755102041


Pass@3 = 0.3469387755102041



pass_at_k,▁
pass_at_k,0.34694


wandb: Agent Starting Run: vng09c9h with config:
wandb: 	do_sample: True
wandb: 	k: 3
wandb: 	max_length_after_input: 250
wandb: 	min_length: 1
wandb: 	num_return_sequences: 4
wandb: 	temperature: 0.1
wandb: 	top_k: 50
wandb: 	top_p: 0.2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TESTING STARTED
@sample 5 -> Pass@3 = 0.4
@sample 10 -> Pass@3 = 0.2
@sample 15 -> Pass@3 = 0.2857142857142857
@sample 20 -> Pass@3 = 0.2631578947368421
@sample 25 -> Pass@3 = 0.3333333333333333
@sample 30 -> Pass@3 = 0.3793103448275862
@sample 35 -> Pass@3 = 0.35294117647058826
@sample 40 -> Pass@3 = 0.38461538461538464
@sample 45 -> Pass@3 = 0.38636363636363635
@sample 50 -> Pass@3 = 0.3469387755102041


Pass@3 = 0.3469387755102041



pass_at_k,▁
pass_at_k,0.34694
